In [639]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import myslack
import datetime as dt
import pickle
import time
import re
from tqdm import trange

In [664]:
driver = webdriver.Chrome()  
url = 'http://produce48.mnet.com/pc/profile/0'
driver.get(url) 

In [839]:
review_count = int(driver.find_element_by_xpath('//*[@id="comment"]/div/h2/strong').text)
comment_page_count = int(review_count/20)+1
review_count, comment_page_count

(3112, 156)

In [840]:
def open_all_comment():
    print('이전 댓글 모두 불러오기 시작...')    
    for _ in trange(comment_page_count):
        try:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/a').click()
            time.sleep(0.3)
        except Exception as e:
            print(e)
            time.sleep(1)
            pass
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    print('이전 댓글 모두 불러오기 성공!')

In [841]:
def make_produce48_df(trainee_name):
    print(trainee_name+'의 댓글 수: '+str(review_count))
    open_all_comment()
    
    print('making dataframe...')
    df = pd.DataFrame(columns=['User_ID','Comment','Date'])
    for i in trange(review_count):
        try:
            df.loc[len(df)] = { 
            'User_ID':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/strong'.format(i+1)).text,
            'Comment':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/p'.format(i+1)).text,
            'Date':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/span'.format(i+1)).text.split(' ')[0],
            }
        except Exception as e:
            print(e)
            pass
    print('making dataframe done!')
    return df

In [842]:
def make_df_name():
    eng_name = \
    re.sub("[^a-zA-Z]"," ",driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[1]/dl[2]/dt/p[2]').text).strip()
    result = eng_name.replace(' ','_')
    return result

name = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[1]/dl[2]/dt/p[2]/span').text
agency = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/h4/span').text

make_df_name(), name, agency

('LEE_HA_EUN', '이하은', 'MNH')

In [ ]:
%time LEE_HA_EUN = make_produce48_df(name)

  0%|          | 0/156 [00:00<?, ?it/s]

이하은의 댓글 수: 3112
이전 댓글 모두 불러오기 시작...


 24%|██▍       | 38/156 [01:18<04:03,  2.07s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="comment"]/div/div[2]/div/a"}
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.13.6 x86_64)



 25%|██▌       | 39/156 [01:19<03:58,  2.04s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="comment"]/div/div[2]/div/a"}
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.13.6 x86_64)



  0%|          | 0/3112 [00:00<?, ?it/s]

이전 댓글 모두 불러오기 성공!
making dataframe...


  4%|▍         | 140/3112 [00:56<19:50,  2.50it/s]

In [ ]:
LEE_HA_EUN.shape

In [ ]:
LEE_HA_EUN.tail()

In [ ]:
pickle.dump(LEE_HA_EUN, open('./produce48/MNH/LEE_HA_EUN.pkl','wb'))

In [ ]:
LEE_HA_EUN_df = pickle.load(open('./produce48/MNH/LEE_HA_EUN.pkl','rb'))

In [ ]:
myslack.send_slack('{} finish!'.format(make_df_name()))

In [ ]:
LEE_HA_EUN_df.tail()

In [663]:
driver.close() # 창닫기
driver.quit() # 브라우져 닫기